# Library importing

In [9]:
!pip install xgboost
!pip install --upgrade install
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


# data pre-proccessing

## data reading

In [ ]:
final_data = pd.read_csv("/kaggle/input/data-cleaning/preprocessing")
final_data.drop(columns='Unnamed: 0',inplace =True)

In [ ]:
final_data.head(3)

In [12]:
preprocessor=joblib.load("/kaggle/input/pre-processing-pipeline/other/default/1/preprocessing_pipeline.pkl")

## Data spliting

In [ ]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [ ]:
x=preprocessor.fit_transform(x)

In [ ]:
model = XGBRegressor(
    bjective = 'reg:squarederror',
    booster =  "gbtree",
    n_estimator = 500 ,
    max_depth = 20 ,
    random_state = 42 ,
    learning_rate = 0.01 ,
    eval_metric = 'logloss',
    early_stopping_rounds = 10
)

In [ ]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 42)
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)

In [ ]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

In [ ]:
model.score(x_train,y_train) 

In [ ]:
model.score(x_test,y_test)